In [1]:
import requests
import pandas as pd
from urllib.request import Request, urlopen
from sodapy import Socrata
import json
import re
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import ipywidgets as wd
from ipywidgets import interact
import seaborn as sns
#import ptitprince as pt
import base64

In [2]:
with open("newSecret.json", "r") as f:
    config = json.load(f)

token = config["TOKEN_API_OD"]
user=config["user"]
passwd=config["pw"]
client_id=config["clientId"]
client_secret=config["clientSecret"]

## MAIN ACCESS FUNCTION

In [3]:
def get_access_token(client_id, client_secret):
    # URL para obtener el token de acceso
    auth_url = 'https://accounts.spotify.com/api/token'
    
    # Creación del encabezado de autorización con client_id y client_secret codificados en base64
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    
    # Datos necesarios para la petición POST
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    
    # Realizar la petición POST
    auth_response = requests.post(auth_url, data=auth_data, headers=auth_header)
    
    # Extraer la respuesta y obtener el token de acceso
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    
    # Devolver el token de acceso
    return access_token


## CALL ME BY YOUR NAME

In [4]:
def get_artists_by_name(query, access_token, limit=50):
    search_url = 'https://api.spotify.com/v1/search'
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    params = {
        'q': f"name:\"{query}\"",
        'type': 'artist',
        'limit': limit
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    
    if 'artists' in response_data:
        return [artist['id'] for artist in response_data['artists']['items']]
    else:
        print("Error en la respuesta de la API:", response_data)
        return []

In [5]:
access_token = get_access_token(client_id, client_secret)

In [6]:
def extract_artists_to_dataframe(response_data):
    # Lista para almacenar datos de artistas
    artist_list = []

    # Iterar sobre cada artista en la respuesta
    for artist in response_data['artists']:
        # Extraer la información relevante
        artist_info = {
            'Name': artist['name'],
            'Genres': ', '.join(artist['genres']),
            'Popularity': artist['popularity'],
            'Spotify_URL': artist['external_urls']['spotify'],
            'Followers': artist['followers']['total'],
            'Image_URL': artist['images'][0]['url'] if artist['images'] else None, # Toma la primera imagen, si existe
            'Artist_ID': artist['id'],
            'Artist_Type': artist['type'],
            'Artist_URI': artist['uri']
        }
        # Agregar a la lista
        artist_list.append(artist_info)

    # Convertir lista en DataFrame
    df_artists = pd.DataFrame(artist_list)

    return df_artists

In [7]:
def get_artists_from_spotify(token, artist_ids):
    """
    Fetch artist details from Spotify's API.
    
    :param access_token: Token for API authorization.
    :param artist_ids: List of artist IDs.
    :return: Response data as a dictionary.
    """
    
    # Endpoint de la API
    base_url = "https://api.spotify.com/v1/artists"
    
    # Headers de la petición (incluye el token de acceso para autorización)
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Parámetros de la petición (IDs de los artistas)
    params = {
        "ids": ",".join(artist_ids)
    }
    
    # Realizar la petición GET a la API de Spotify
    response = requests.get(base_url, headers=headers, params=params)
    
    # Convertir la respuesta en formato JSON a un diccionario de Python
    response_data = response.json()

    return response_data

""" Ejemplo de uso:

artist_ids = ["2CIMQHirSU0MQqyYHq0eOx", "57dN52uHvrHOxijzpIgu3E"]  # Reemplaza con los IDs de los artistas que quieras consultar
token=access_token
 Obtener respuesta de la API
response_data = get_artists_from_spotify(token, artist_ids)

# Procesar la respuesta con la función extract_artists_to_dataframe
df_artist = extract_artists_to_dataframe(response_data)
#print(df)
df_artist"""

' Ejemplo de uso:\n\nartist_ids = ["2CIMQHirSU0MQqyYHq0eOx", "57dN52uHvrHOxijzpIgu3E"]  # Reemplaza con los IDs de los artistas que quieras consultar\ntoken=access_token\n Obtener respuesta de la API\nresponse_data = get_artists_from_spotify(token, artist_ids)\n\n# Procesar la respuesta con la función extract_artists_to_dataframe\ndf_artist = extract_artists_to_dataframe(response_data)\n#print(df)\ndf_artist'

## THE INPUT

In [8]:
Spotify_List_Id='7M36AGwbD9OpGUZN3IWx7V'
#7M36AGwbD9OpGUZN3IWx7V

In [9]:
def get_band_from_playlist_id(token, playlist_id):
    """
    Fetch band names from a Spotify playlist.
    
    :param token: Token for API authorization.
    :param playlist_id: Spotify Playlist ID.
    :return: List of band names.
    """
    playlist_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(playlist_url, headers=headers)
    
    if response.status_code == 200:
        playlist_data = response.json()
        bands = []
        for item in playlist_data['items']:
            # Cada 'item' contiene información de una pista, incluyendo sus artistas
            track_artists = item['track']['artists']
            for artist in track_artists:
                bands.append(artist['name'])
        return bands
    else:
        print(f"Error fetching playlist data: {response.status_code}")
        return []

# Uso de la función
# token = "tu_token_de_acceso"
# playlist_id = "id_de_tu_playlist"
# band_names = get_band_from_playlist_id(token, playlist_id)
# print(band_names)


In [10]:
band_names = get_band_from_playlist_id(access_token, Spotify_List_Id)

In [11]:
Band_Names=pd.DataFrame(band_names,
columns=['Band_Name'])
Band_Names

,Band_Name
0,Deformation
1,SYRACUSÆ
2,Jose Macario
3,Under Threat
4,Demonsquest
...,...
102,Epiphany.
103,Licantropia
104,ubergehen
105,Cosmology X


In [12]:
Band_Names.columns

Index(['Band_Name'], dtype='object')

In [13]:
test_name=band_names[1]
named_test = get_artists_by_name(test_name, access_token)
named_test

['65uP1jjzWh4VISZCA4s3X7',
 '3tPNUZKUf9Pa8R84yWpSvO',
 '2tJE40GixcEoh9zwoBD07H',
 '3s5NSrjy5sXASsWsm55iPy',
 '64iLtTMfnTAuuMbRk66boJ',
 '4LOPGgD3DBWffxYypHDzry',
 '5AB3ONEZzBBSvjYYStYgap',
 '6xinS4Wgb422R7KrrJ9UYU',
 '5JDM5S7eyY3ypNOyS5e0ML',
 '6WjPUWcR55uboKEjvH1Pt7',
 '0axxjRxJBx2ciiPYRcBPJ2',
 '4JmS6FBtKxZ9sQwS4Ezk4j',
 '0I4flbOXVYUts2wY3cPt1R',
 '4ZiIj8ThbKaep4Kc0V5xbZ',
 '2LXgEluAGEdhhBqTYtve0g',
 '1mOK3QVWe6wabndtqZGERg',
 '4lqBkVmOyYfnDQCm5mA5ko',
 '245qp5aDyBGuhSFu8GGSJi',
 '59mntshH5MOPANbk9f69hP',
 '4TSO4eUt8kze7Ac5Gg1y8e',
 '3ntNlXR1aenb9hNEL02WYH',
 '2rD1kXQRKt6KpmKnLAZZ2E',
 '0KIxr4HfjzUEBKy6vlfR9O',
 '08cAZWaoVZnCrDCDY6P7tk',
 '7sUmD0wgjQR7cavOCCdP5W',
 '0hKhw4tF73tq6390kUouxM',
 '1qo7mKjITlrCOs9nptnPEb',
 '2JTmMIiuyMwoO9nvPoZJ2Z',
 '4XZ4x9dUVCLnrm8oRQMlDt']

In [14]:
token=access_token
response_data_names_test = get_artists_from_spotify(token, named_test)
df_metal_artist_named_test = extract_artists_to_dataframe(response_data_names_test)
Name_test=band_names[1]
locked_test = df_metal_artist_named_test.loc[df_metal_artist_named_test['Name'] == Name_test, 'Artist_ID']
locked_test

3    3s5NSrjy5sXASsWsm55iPy
Name: Artist_ID, dtype: object

In [15]:
test_name

'SYRACUSÆ'

In [16]:
Name = band_names[1]
print("Buscando:", Name)  # Esto te mostrará qué estás buscando exactamente

# Verifica si el nombre existe en el DataFrame
print(Name in df_metal_artist_named_test['Name'].values)




Buscando: SYRACUSÆ
True


In [20]:
#token=access_token
BAND_NAMES_LIST=[]
for name in band_names:
    #TEST_NAME=band_names[i]
    NAMED_TEST = get_artists_by_name(name, access_token)
    RESPONSE_DATA_NAMES = get_artists_from_spotify(access_token, NAMED_TEST)
    DF_METAL_ARTIST_NAMES = extract_artists_to_dataframe(RESPONSE_DATA_NAMES)
    LOCKED_TEST = DF_METAL_ARTIST_NAMES.loc[DF_METAL_ARTIST_NAMES['Name'] == name, 'Artist_ID']
    BAND_NAMES_LIST.append(LOCKED_TEST)
    bands_DF=pd.DataFrame(BAND_NAMES_LIST)

In [21]:
bands_DF

,1,27,25,3,29,11,7,23,9,42,13,5,15
Artist_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist_ID,7HiasoqcVt5qyJtcCbI2bM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist_ID,763E4V9H0tCY8SQvqKDt7C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Artist_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artist_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
band_id_list = []

for name in band_names:
    # Obtener IDs de la banda por nombre
    named_test = get_artists_by_name(name, access_token)
    
    # Obtener detalles del artista a partir del ID
    response_data_names = get_artists_from_spotify(access_token, named_test)
    df_metal_artist_names = extract_artists_to_dataframe(response_data_names)
    
    # Buscar la coincidencia exacta con el nombre y obtener el ID
    locked_test = df_metal_artist_names.loc[df_metal_artist_names['Name'] == name, 'Artist_ID']
    
    # Si existe un ID, añadir al listado
    if not locked_test.empty:
        artist_id = locked_test.iloc[0]
        band_id_list.append({'Name': name, 'Artist_ID': artist_id})

# Crear un DataFrame a partir de la lista de diccionarios
band_id_df = pd.DataFrame(band_id_list)

# Mostrar el DataFrame
print(band_id_df)


                   Name               Artist_ID
0          Jose Macario  7HiasoqcVt5qyJtcCbI2bM
1          Under Threat  763E4V9H0tCY8SQvqKDt7C
2       Ground of Chaos  6VbDueRRIISqEPIuG8VnXK
3       Ground of Chaos  6VbDueRRIISqEPIuG8VnXK
4          Under Threat  763E4V9H0tCY8SQvqKDt7C
5          Under Threat  763E4V9H0tCY8SQvqKDt7C
6          Under Threat  763E4V9H0tCY8SQvqKDt7C
7          Under Threat  763E4V9H0tCY8SQvqKDt7C
8     Absolution Denied  3J25kteCsA6g3SK359uwA5
9                  Vein  19BL3j9p5vtcL29joapf0E
10  Poison the Preacher  3InxEfnjMqTJWqsWvHjjPU
11  Poison the Preacher  3InxEfnjMqTJWqsWvHjjPU
12         Ennui Bogotá  3UxjBvP0pvz35UevSs25eh
13         Ennui Bogotá  3UxjBvP0pvz35UevSs25eh
14         Ennui Bogotá  3UxjBvP0pvz35UevSs25eh
15        Eduardo Prado  7LbtcUOskCbjhcYpSuWBaq
16            Sin Pudor  70G6m8FrCDxGQoJdIC94aX
17            Sin Pudor  70G6m8FrCDxGQoJdIC94aX
18         Paola Camelo  1sjucRVtQ7Gb18wFiQ9fFr
19      Viviana Cabrera  362lruSCnUoEs8W

In [23]:
band_id_df

,Name,Artist_ID
0,Jose Macario,7HiasoqcVt5qyJtcCbI2bM
1,Under Threat,763E4V9H0tCY8SQvqKDt7C
2,Ground of Chaos,6VbDueRRIISqEPIuG8VnXK
3,Ground of Chaos,6VbDueRRIISqEPIuG8VnXK
4,Under Threat,763E4V9H0tCY8SQvqKDt7C
5,Under Threat,763E4V9H0tCY8SQvqKDt7C
6,Under Threat,763E4V9H0tCY8SQvqKDt7C
7,Under Threat,763E4V9H0tCY8SQvqKDt7C
8,Absolution Denied,3J25kteCsA6g3SK359uwA5
9,Vein,19BL3j9p5vtcL29joapf0E
